===============================================================
Step 0: Setup and Authentication
===============================================================
We initialize both BigQuery and Vertex AI clients to interact with our data and AI services.
- BigQuery client is used to query the news corpus and sanctions tables.
- Vertex AI client is used for ML embeddings via BigQuery ML.
The credentials JSON file is loaded to authenticate the session.

In [ ]:
from google.cloud import aiplatform
from google.cloud import bigquery
import pandas as pd
import numpy as np
from time import sleep
import os
import matplotlib.pyplot as plt

--------------------------------------------------------------
Step 1: Set up BigQuery client
--------------------------------------------------------------
We initialize the BigQuery client with our project ID.
This allows us to run queries against our BigQuery datasets, including the news corpus and sanctions tables we created.

In [ ]:
# Path to your uploaded JSON key
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "..."

# Initialize BigQuery client (dataset in US region)
bq_client = bigquery.Client(project='kagglecov1', location='US')

# Initialize Vertex AI client (supported US region)
aiplatform.init(project='kagglecov1', location='us-central1')

print("✅ Authentication successful")


In [ ]:
# List tables in your dataset to confirm access
tables = list(bq_client.list_tables("Sanctions1"))
for table in tables:
    print(table.table_id)


===============================================================
Step 2: Pull Entity Matches from BigQuery
===============================================================
This query retrieves all entities extracted from news articles (GDELT) that have been matched against sanctioned entities. 
Columns explained:
- entity_norm: normalized entity name from news
- sanction_id: ID of the matching sanction record
- sanction_name: name of the sanctioned entity
- match_type: 'exact' or 'vector' (semantic match)
- distance: semantic similarity (lower = more similar)

In [ ]:
# Replace with your GCP project ID
project_id = "kagglecov1"

# Create BigQuery client with project
bq_client = bigquery.Client(project=project_id)

# Pull all entity matches
query = """
SELECT entity_norm, sanction_id, sanction_name, match_type, distance
FROM `kagglecov1.Sanctions1.v_gkg_sanctions_hits`
WHERE entity_norm IS NOT NULL
"""
matches_df = bq_client.query(query).to_dataframe()
matches_df.head()

===============================================================
Step 3: Aggregate Maximum Risk per Entity
===============================================================
Groups by entity to determine the highest risk score observed for each entity.
This ensures each entity is represented by its most critical exposure.

In [ ]:
# Risk scoring logic
matches_df['risk_score'] = matches_df.apply(
    lambda row: 1.0 if row['match_type']=='exact' else 1 - row['distance'], axis=1
)

# Aggregate max risk per entity
risk_df = matches_df.groupby('entity_norm')['risk_score'].max().reset_index()

# Top risky entities
risk_df = risk_df.sort_values('risk_score', ascending=False)
risk_df.head(10)

===============================================================
Step 4: Inspect Top Risky Entities
===============================================================
Filters the original matches to only include the top 10 risky entities.
This shows each match's type, similarity distance, and calculated risk score.

In [ ]:
top_risky = matches_df[matches_df['entity_norm'].isin(risk_df.head(10)['entity_norm'])]
top_risky[['entity_norm','match_type','distance','risk_score']]

===============================================================
Step 5: Function to Query Risk for a Specific Entity
===============================================================
get_entity_risk function takes any entity name and returns its maximum risk score.
Handles both exact matches and semantic vector matches.

In [ ]:
def get_entity_risk(entity_name, df):
    entity_name_norm = entity_name.lower().strip()
    
    rows = df[df['entity_norm'] == entity_name_norm]
    
    if rows.empty:
        return 0.0  
    
    risk_scores = []
    for _, row in rows.iterrows():
        if row['match_type'] == 'exact':
            risk_scores.append(1.0)
        elif row['match_type'] == 'vector':
            risk_scores.append(max(0.0, 1 - row['distance']))
    
    return max(risk_scores)

# Example usage
entity_risk_score = get_entity_risk('Vladimir Putin', matches_df)
print(f"Risk score for Vladimir Putin: {entity_risk_score}")